# Мемы

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

In [4]:
path = 'drive/MyDrive/meme'


In [5]:
event = pd.read_csv(path+'/danila_event.csv')


In [6]:
meme = pd.read_csv(path+'/danila_meme.csv')
user = pd.read_csv(path+'/danila_user.csv')

In [7]:
meme.head()

,meme_id,meme_type,source_id,published_at,caption,content_lang,is_available,likes,dislikes,created_at,updated_at,is_broken,content_text,caption_lang,language_id
0,28997,photo,79.0,2020-04-12 01:48:13+00,I mean shitt it ain't their fault,en,True,17,14,2020-04-12 09:04:37.341908+00,2020-10-26 19:20:58.198825+00,False,brandon @brndvx females live on recruit diffic...,en,3.0
1,10873,photo,43.0,2020-03-25 11:48:14+00,NaN,ru,True,79,74,2020-03-25 20:00:01.154096+00,2020-10-26 19:20:58.226846+00,False,"Я не робот Докажите, что всякое односвязное Ит...",ww,2.0
2,9874,photo,78.0,2020-03-24 07:18:03+00,Anime toddies expert,en,True,1,7,2020-03-24 13:06:18.588359+00,2020-10-26 19:20:58.259045+00,False,When you find the actual interview and it does...,en,3.0
3,43458,photo,43.0,2020-05-23 12:50:52+00,NaN,ru,False,8,12,2020-05-24 22:03:34.926116+00,2020-10-26 19:20:58.265179+00,False,Google сколько калорий в грамме урана а Все В ...,ww,2.0
4,9000,photo,76.0,2020-03-23 07:16:05+00,New treasure,und,True,2,9,2020-03-23 13:04:55.258554+00,2020-10-26 19:20:58.29742+00,False,YRO .20.,en,3.0


In [8]:
user.head()

,user_id,deep_link,is_blocked_bot,is_banned,is_moderator,sessions,last_session_memes,watched_memes,created_at,updated_at,interface_lang,gender
0,820326025,anime0ch_lang_ru,False,False,False,6,1,20,2020-09-12 20:07:11.668743+00,2020-10-26 12:16:18.458654+00,ru,M
1,1040875164,likemeupbot_lang_en,False,False,False,2,1,7,2020-07-21 05:54:12.671036+00,2020-10-26 14:37:05.637591+00,en,NaN
2,449432539,debil_ebaniy,True,False,False,2,1,9,2020-09-14 19:02:06.80911+00,2020-10-17 21:26:52.029824+00,ru,F
3,230248850,NaN,False,False,False,26,1,38,2020-03-16 00:36:42.252438+00,2020-10-26 12:11:02.159405+00,ru,M
4,7719424,NaN,True,False,False,27,6,49,2020-03-15 22:30:45.300242+00,2020-10-17 21:26:52.19941+00,ru,M


In [9]:
event.head()

,event_id,reaction_id,session_idx,meme_idx,created_at,updated_at,meme_id,user_id
0,3607617,2.0,55,11,2020-10-24 19:45:50.964474+00,2020-10-24 19:45:51.049752+00,24379,280747086
1,3608100,1.0,280,45,2020-10-24 19:52:33.681933+00,2020-10-24 19:52:33.763993+00,49173,285739883
2,3607629,1.0,55,13,2020-10-24 19:46:08.690407+00,2020-10-24 19:46:08.779079+00,8340,280747086
3,3607649,NaN,30,2,2020-10-24 19:46:24.443014+00,2020-10-24 19:46:24.519787+00,64748,78991522
4,3608045,2.0,35,341,2020-10-24 19:51:28.544856+00,2020-10-24 19:51:28.586788+00,45163,860086058


In [10]:
# Создать рекомендательную систему для каждого пользователя

## Создать рекомендательную систему для каждого пользователя

In [11]:
#meme['is_broken'].value_counts()

meme = meme[meme['is_broken'] == False]

In [12]:
meme

,meme_id,meme_type,source_id,published_at,caption,content_lang,is_available,likes,dislikes,created_at,updated_at,is_broken,content_text,caption_lang,language_id
0,28997,photo,79.0,2020-04-12 01:48:13+00,I mean shitt it ain't their fault,en,True,17,14,2020-04-12 09:04:37.341908+00,2020-10-26 19:20:58.198825+00,False,brandon @brndvx females live on recruit diffic...,en,3.0
1,10873,photo,43.0,2020-03-25 11:48:14+00,NaN,ru,True,79,74,2020-03-25 20:00:01.154096+00,2020-10-26 19:20:58.226846+00,False,"Я не робот Докажите, что всякое односвязное Ит...",ww,2.0
2,9874,photo,78.0,2020-03-24 07:18:03+00,Anime toddies expert,en,True,1,7,2020-03-24 13:06:18.588359+00,2020-10-26 19:20:58.259045+00,False,When you find the actual interview and it does...,en,3.0
3,43458,photo,43.0,2020-05-23 12:50:52+00,NaN,ru,False,8,12,2020-05-24 22:03:34.926116+00,2020-10-26 19:20:58.265179+00,False,Google сколько калорий в грамме урана а Все В ...,ww,2.0
4,9000,photo,76.0,2020-03-23 07:16:05+00,New treasure,und,True,2,9,2020-03-23 13:04:55.258554+00,2020-10-26 19:20:58.29742+00,False,YRO .20.,en,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61851,29440,photo,80.0,2020-04-12 16:52:08+00,She never needed makeup,en,True,2,8,2020-04-12 21:02:06.757845+00,2020-10-26 19:20:58.103871+00,False,When your girlfriend takes off her makeup but ...,en,3.0
61852,50912,photo,18.0,2020-07-18 17:00:07+00,NaN,ru,True,539,225,2020-07-19 06:14:42.821713+00,2020-10-26 19:20:58.113269+00,False,katevictoriax Парней нужно чаще ласкать. Клади...,ww,2.0
61853,22646,photo,2.0,2020-04-06 06:52:16+00,NaN,en,False,1,2,2020-04-06 08:00:00.352386+00,2020-10-26 19:20:58.143725+00,False,I COME TOTHEDARKSIDE WE HAVE RESPIRATOR MASKS,ww,3.0
61854,49822,photo,NaN,2020-07-10 12:08:55+00,NaN,ru,True,5,17,2020-07-10 18:15:20.413744+00,2020-10-26 19:20:58.148913+00,False,"Шаг Не знаете, куда поехать? Едьте в центр! Ян...",ww,2.0


In [13]:
user['is_banned'].value_counts()

False    19918
Name: is_banned, dtype: int64

In [14]:
# Проверить гипотезу: англ пользователи более активные чем русские
# лайки
# дизлайки
# отношение L/D
# Диаграмма (рус в среднем и eng в среднем)

In [15]:
user['interface_lang'].value_counts()

ru    14156
en     5714
Name: interface_lang, dtype: int64

In [16]:
reactions = event.groupby('user_id').count()['event_id']
reactions

user_id
1615           47
3859          193
12199           3
13934           6
14229          15
             ... 
1399229617      1
1399329801      1
1399419434     11
1399511615      2
1399599499      9
Name: event_id, Length: 19699, dtype: int64

In [26]:
# 1 - лайк
# 2 - дизлайк
likes = event[event['reaction_id'] == 1].groupby('user_id').count()['event_id']
#likes.rename(columns = {'event_id':'likes'})
likes


user_id
1615           25
3859          106
13934           1
14229           2
16060          14
             ... 
1398188580      5
1398529159     12
1398799657      6
1399419434      6
1399599499      8
Name: event_id, Length: 16633, dtype: int64

In [25]:
dislikes =  event[event['reaction_id'] == 2].groupby('user_id').count()['event_id']
dislikes


user_id
1615           11
3859           83
14229          10
16060         111
20535          10
             ... 
1397824939      2
1397992921      5
1397993276      3
1398022998     14
1398799657      1
Name: event_id, Length: 15881, dtype: int64

In [27]:
event_reactions = pd.DataFrame([likes.values,dislikes.values, reactions.values]).transpose()
event_reactions.rename(columns = {0:'likes', 1:'dislikes', 2:'reactions'}, inplace = True)


In [28]:
event_reactions.index = reactions.index
event_reactions

,likes,dislikes,reactions
user_id,,,
1615,25.0,11.0,47.0
3859,106.0,83.0,193.0
12199,1.0,10.0,3.0
13934,2.0,111.0,6.0
14229,14.0,10.0,15.0
...,...,...,...
1399229617,NaN,NaN,1.0
1399329801,NaN,NaN,1.0
1399419434,NaN,NaN,11.0


### Построить график для лайков

In [29]:
likes_users_ids = likes.index
likes_users_ids

Int64Index([      1615,       3859,      13934,      14229,      16060,
                 20535,      28327,      30767,      37459,      39129,
            ...
            1397540701, 1397824939, 1397992921, 1397993276, 1398022998,
            1398188580, 1398529159, 1398799657, 1399419434, 1399599499],
           dtype='int64', name='user_id', length=16633)

In [30]:
users_likes = user[user.user_id.isin(likes_users_ids)].set_index('user_id')['interface_lang'].copy()
users_likes

user_id
820326025     ru
1040875164    en
449432539     ru
1008072970    en
8239465       ru
              ..
125474047     ru
608727447     ru
1254871249    ru
184147840     ru
761418142     ru
Name: interface_lang, Length: 16633, dtype: object

In [31]:
detailed_likes =  pd.DataFrame([users_likes, likes]).transpose()
grouped_det_likes = detailed_likes.groupby('interface_lang').count()
grouped_det_likes

,event_id
interface_lang,
en,4546
ru,12087


In [32]:
import plotly.express as px
fig = px.bar(grouped_det_likes, x = grouped_det_likes.index, y = 'event_id')
fig.show()

### Для каждого source соотношение лайков дизлайков

In [33]:
memes_grouped = meme[['source_id', 'likes', 'dislikes']]
memes_grouped_sum = memes_grouped.groupby('source_id').sum()
memes_grouped_sum['l\\d'] = memes_grouped_sum['likes'] / memes_grouped_sum['dislikes']
memes_grouped_sum

,likes,dislikes,l\d
source_id,,,
1.0,141,447,0.315436
2.0,8753,11876,0.737033
3.0,17852,19303,0.924830
4.0,17265,27841,0.620129
5.0,11334,15323,0.739672
...,...,...,...
155.0,172,463,0.371490
156.0,741,1331,0.556724
157.0,823,1532,0.537206


### Проделать ту же работу для дизлайков и всех реакций

#### Дизлайки

In [38]:
dislikes_users_ids = dislikes.index
dislikes_users_ids

Int64Index([      1615,       3859,      14229,      16060,      20535,
                 28327,      30767,      37459,      45215,      45795,
            ...
            1396154939, 1396196160, 1397068897, 1397208607, 1397490541,
            1397824939, 1397992921, 1397993276, 1398022998, 1398799657],
           dtype='int64', name='user_id', length=15881)

In [39]:
users_dislikes = user[user.user_id.isin(dislikes_users_ids)].set_index('user_id')['interface_lang'].copy()
users_dislikes

user_id
820326025     ru
1040875164    en
7719424       ru
8239465       ru
379260363     ru
              ..
125474047     ru
608727447     ru
1254871249    ru
184147840     ru
761418142     ru
Name: interface_lang, Length: 15881, dtype: object

In [40]:
detailed_dislikes =  pd.DataFrame([users_dislikes, dislikes]).transpose()
grouped_det_dislikes = detailed_dislikes.groupby('interface_lang').count()
grouped_det_dislikes

,event_id
interface_lang,
en,3532
ru,12349


In [41]:
fig = px.bar(grouped_det_dislikes, x = grouped_det_dislikes.index, y = 'event_id')
fig.show()

### Все реакции

In [42]:
reactions_users_ids = reactions.index
reactions_users_ids

Int64Index([      1615,       3859,      12199,      13934,      14229,
                 16060,      20535,      28327,      30767,      37459,
            ...
            1398022998, 1398188580, 1398529159, 1398678076, 1398799657,
            1399229617, 1399329801, 1399419434, 1399511615, 1399599499],
           dtype='int64', name='user_id', length=19699)

In [43]:
users_reactions = user[user.user_id.isin(reactions_users_ids)].set_index('user_id')['interface_lang'].copy()
users_reactions

user_id
820326025     ru
1040875164    en
449432539     ru
230248850     ru
7719424       ru
              ..
125474047     ru
608727447     ru
1254871249    ru
184147840     ru
761418142     ru
Name: interface_lang, Length: 19699, dtype: object

In [44]:
detailed_reactions = pd.DataFrame([users_reactions, reactions]).transpose()
grouped_det_reactions = detailed_reactions.groupby('interface_lang').count()

In [45]:
fig = px.bar(grouped_det_reactions, x = grouped_det_reactions.index, y = 'event_id')
fig.show()

In [46]:
grouped_det_likes

,event_id
interface_lang,
en,4546
ru,12087


In [47]:
grouped_det_dislikes

,event_id
interface_lang,
en,3532
ru,12349


In [48]:
grouped_det_likes / grouped_det_dislikes

,event_id
interface_lang,
en,1.287089
ru,0.978784


In [49]:
grouped_det_reactions

,event_id
interface_lang,
en,5684
ru,14015


Выводы, которые можно сделать из проделанной работы из имеющихся данных:
- В наблюдении кол-во людей с RU интерфейсом преобладает над иными значениями в ~3 раза, но это не означает, что они более активны
- Люди с EN интерфейсом предпочитают лайкать мемы (L/D показатель = 1.28 для EN и ~0.98 для RU)

# ДЗ
- Построить гипотезы (2-3 штуки)
- Датасет + модель (какой мем какому пользователю зайдет)
- Реком система в цикле(как вариант) будет смотреть каждый мем и классифицировать на 1 или 0
- Презентация

In [50]:
meme.meme_type.value_counts()

photo        55917
animation     2885
video         2714
Name: meme_type, dtype: int64

## Гипотеза №1: отношение l/d у мужчин ниже, чем у женщин

In [56]:
data_for_gender = event[['reaction_id', 'user_id']].copy()

In [57]:
data_for_gender

,reaction_id,user_id
0,2.0,280747086
1,1.0,285739883
2,1.0,280747086
3,NaN,78991522
4,2.0,860086058
...,...,...
3652698,2.0,860086058
3652699,2.0,381510059
3652700,2.0,285739883
3652701,1.0,245482


In [58]:
users = user[['user_id', 'gender']].copy()

In [59]:
users

,user_id,gender
0,820326025,M
1,1040875164,NaN
2,449432539,F
3,230248850,M
4,7719424,M
...,...,...
19913,184147840,NaN
19914,663303657,NaN
19915,761418142,NaN
19916,669142397,NaN


In [54]:
users['gender'].value_counts()

M    2679
N    1927
F    1109
Name: gender, dtype: int64

In [60]:
inner = pd.merge(users, data_for_gender, on='user_id')

In [70]:
users_likes_w_gender = inner[inner['reaction_id'] == 1].copy()
users_dislikes_w_gender = inner[inner['reaction_id'] == 2].copy()


In [71]:
users_likes_w_gender

,user_id,gender,reaction_id
15,820326025,M,1.0
16,820326025,M,1.0
17,820326025,M,1.0
22,820326025,M,1.0
24,1040875164,NaN,1.0
...,...,...,...
3652689,761418142,NaN,1.0
3652690,761418142,NaN,1.0
3652692,761418142,NaN,1.0
3652694,761418142,NaN,1.0


In [83]:
M_likes = users_likes_w_gender[users_likes_w_gender['gender'] == 'M'].copy()
F_likes = users_likes_w_gender[users_likes_w_gender['gender'] == 'F'].copy()
M_dislikes = users_dislikes_w_gender[users_dislikes_w_gender['gender'] == 'M'].copy()
F_dislikes = users_dislikes_w_gender[users_dislikes_w_gender['gender'] == 'F'].copy()

In [92]:
MF_values_dict = {'M/F_ratio':[len(M_likes) / len (M_dislikes), len(F_likes) / len (F_dislikes)]}
MF_values = pd.DataFrame(MF_values_dict)
MF_values.rename(index={0:'M',1:'F'}, inplace = True)
MF_values

,M/F_ratio
M,1.337439
F,1.653884


Как и предполагалось, L/D отношение у мужчин ниже, чем у женщин. Возможно, это связано с тем, что мужчины в целом как юзеры более активны чем женщины. Возможно, мужчины в Интернете более категоричны и при недовольстве чаще ставят дизлайк, когда представительницы женского пола просто проигнорируют и пойдут дальше  

## Гипотеза №2: фото мемы в среднем лайкают больше, чем мемы иного типа

In [94]:
meme.meme_type.value_counts()

photo        55917
animation     2885
video         2714
Name: meme_type, dtype: int64

In [97]:
detailed_meme = meme[['meme_type', 'likes', 'dislikes']].copy()
detailed_meme.groupby('meme_type').mean()

,likes,dislikes
meme_type,,
animation,14.407972,21.242288
photo,33.899691,28.570274
video,12.654753,23.306559


Как и предполагалось, в среднем из 3 типов мемов, большее кол-во лайков собирают мемы-картинки, а не мемы в виде видео или анимации. Это можно объяснить тем, что юзеру удобнее(быстрее) посмотреть картинку, и ему не нужно тратить много времени на ее просмотр, поэтому и лайк проще получить картинке. 

## Датасет + модель (какой мем какому пользователю зайдет)

* Подготовка данных

In [317]:
event.head()

,event_id,reaction_id,session_idx,meme_idx,created_at,updated_at,meme_id,user_id
0,3607617,2.0,55,11,2020-10-24 19:45:50.964474+00,2020-10-24 19:45:51.049752+00,24379,280747086
1,3608100,1.0,280,45,2020-10-24 19:52:33.681933+00,2020-10-24 19:52:33.763993+00,49173,285739883
2,3607629,1.0,55,13,2020-10-24 19:46:08.690407+00,2020-10-24 19:46:08.779079+00,8340,280747086
3,3607649,NaN,30,2,2020-10-24 19:46:24.443014+00,2020-10-24 19:46:24.519787+00,64748,78991522
4,3608045,2.0,35,341,2020-10-24 19:51:28.544856+00,2020-10-24 19:51:28.586788+00,45163,860086058


In [318]:
meme.head()

,meme_id,meme_type,source_id,published_at,caption,content_lang,is_available,likes,dislikes,created_at,updated_at,is_broken,content_text,caption_lang,language_id
0,28997,photo,79.0,2020-04-12 01:48:13+00,I mean shitt it ain't their fault,en,True,17,14,2020-04-12 09:04:37.341908+00,2020-10-26 19:20:58.198825+00,False,brandon @brndvx females live on recruit diffic...,en,3.0
1,10873,photo,43.0,2020-03-25 11:48:14+00,NaN,ru,True,79,74,2020-03-25 20:00:01.154096+00,2020-10-26 19:20:58.226846+00,False,"Я не робот Докажите, что всякое односвязное Ит...",ww,2.0
2,9874,photo,78.0,2020-03-24 07:18:03+00,Anime toddies expert,en,True,1,7,2020-03-24 13:06:18.588359+00,2020-10-26 19:20:58.259045+00,False,When you find the actual interview and it does...,en,3.0
3,43458,photo,43.0,2020-05-23 12:50:52+00,NaN,ru,False,8,12,2020-05-24 22:03:34.926116+00,2020-10-26 19:20:58.265179+00,False,Google сколько калорий в грамме урана а Все В ...,ww,2.0
4,9000,photo,76.0,2020-03-23 07:16:05+00,New treasure,und,True,2,9,2020-03-23 13:04:55.258554+00,2020-10-26 19:20:58.29742+00,False,YRO .20.,en,3.0


In [319]:
user.head()

,user_id,deep_link,is_blocked_bot,is_banned,is_moderator,sessions,last_session_memes,watched_memes,created_at,updated_at,interface_lang,gender
0,820326025,anime0ch_lang_ru,False,False,False,6,1,20,2020-09-12 20:07:11.668743+00,2020-10-26 12:16:18.458654+00,ru,M
1,1040875164,likemeupbot_lang_en,False,False,False,2,1,7,2020-07-21 05:54:12.671036+00,2020-10-26 14:37:05.637591+00,en,NaN
2,449432539,debil_ebaniy,True,False,False,2,1,9,2020-09-14 19:02:06.80911+00,2020-10-17 21:26:52.029824+00,ru,F
3,230248850,NaN,False,False,False,26,1,38,2020-03-16 00:36:42.252438+00,2020-10-26 12:11:02.159405+00,ru,M
4,7719424,NaN,True,False,False,27,6,49,2020-03-15 22:30:45.300242+00,2020-10-17 21:26:52.19941+00,ru,M


In [320]:
# Создал копии датасетов, чтобы не изменять начальные файлы
event_df = event.copy()
meme_df = meme.copy()
user_df = user.copy()

In [321]:
#Удаление данных, которые неинформативны в рамках исследования
event_df.drop(['session_idx', 'meme_idx', 'created_at', 'updated_at', 'event_id'], axis = 1, inplace = True)

In [322]:
meme_df.columns

Index(['meme_id', 'meme_type', 'source_id', 'published_at', 'caption',
       'content_lang', 'is_available', 'likes', 'dislikes', 'created_at',
       'updated_at', 'is_broken', 'content_text', 'caption_lang',
       'language_id'],
      dtype='object')

In [323]:
#Удаление данных, которые неинформативны в рамках исследования
meme_df.drop(['published_at', 'caption','created_at', 'updated_at','content_text'], axis = 1, inplace = True)

In [324]:
user_df.columns

Index(['user_id', 'deep_link', 'is_blocked_bot', 'is_banned', 'is_moderator',
       'sessions', 'last_session_memes', 'watched_memes', 'created_at',
       'updated_at', 'interface_lang', 'gender'],
      dtype='object')

### Работа с user_df

In [325]:
#Удаление данных, которые неинформативны в рамках исследования (тут хотел бы отметить, что поле gender не хотелось удалять, но по нему нет данных по большинству записей, поэтому был вынужден удалить это поле)
user_df.drop(['deep_link','last_session_memes', 'created_at', 'updated_at', 'is_banned','is_moderator', 'gender', 'is_blocked_bot'], axis = 1, inplace = True)

In [326]:
user_df.interface_lang.replace({'en':1, 'ru':0}, inplace = True) # 1 если интерфейс английский, 0 если русский

In [327]:
user_df.interface_lang.value_counts() # ~75% используют русский интерфейс

0.0    14156
1.0     5714
Name: interface_lang, dtype: int64

In [328]:
user_df.isna().sum()

user_id            0
sessions           0
watched_memes      0
interface_lang    48
dtype: int64

In [329]:
# Конечно, 48 nan можно было запихнуть в любую категорию, т.к. малое число пропусков, но захотел распределить пропорционально
import random
nans = user_df['interface_lang'].isna()
length = sum(nans)
replacement = random.choices([0, 1], weights=[.75, .25], k=length)
user_df.loc[nans,'interface_lang'] = replacement
user_df.isna().sum()

user_id           0
sessions          0
watched_memes     0
interface_lang    0
dtype: int64

In [330]:
user_df.interface_lang.value_counts() # Не получилось :)

0.0    14191
1.0     5727
Name: interface_lang, dtype: int64

In [331]:
user_df.head() # отформатированный датасэт

,user_id,sessions,watched_memes,interface_lang
0,820326025,6,20,0.0
1,1040875164,2,7,1.0
2,449432539,2,9,0.0
3,230248850,26,38,0.0
4,7719424,27,49,0.0


### Работа с event_df

In [332]:
event_df.head()

,reaction_id,meme_id,user_id
0,2.0,24379,280747086
1,1.0,49173,285739883
2,1.0,8340,280747086
3,NaN,64748,78991522
4,2.0,45163,860086058


In [333]:
event_df = event_df.drop(event_df[(event_df.reaction_id == 3) | (event_df.reaction_id == 4) |(event_df.reaction_id == 5) ].index) # Удаление действий админов

In [334]:
event_df.isna().sum()

reaction_id    41960
meme_id            0
user_id            0
dtype: int64

In [335]:
event_df.reaction_id.value_counts()

1.0    1866242
2.0    1611488
Name: reaction_id, dtype: int64

In [336]:
nans = event_df['reaction_id'].isna()  # пропорциональное распределение лайков/дизлайков по nan значениям
length = sum(nans)
replacement = random.choices([1, 2], weights=[.5, .5], k=length)
event_df.loc[nans,'reaction_id'] = replacement
event_df.isna().sum()

reaction_id    0
meme_id        0
user_id        0
dtype: int64

In [337]:
event_df.reaction_id.value_counts()

1.0    1887234
2.0    1632456
Name: reaction_id, dtype: int64

In [338]:
event_df.head() # отформатированный датасэт

,reaction_id,meme_id,user_id
0,2.0,24379,280747086
1,1.0,49173,285739883
2,1.0,8340,280747086
3,2.0,64748,78991522
4,2.0,45163,860086058


### Работа с meme_df

In [339]:
meme_df.head()

,meme_id,meme_type,source_id,content_lang,is_available,likes,dislikes,is_broken,caption_lang,language_id
0,28997,photo,79.0,en,True,17,14,False,en,3.0
1,10873,photo,43.0,ru,True,79,74,False,ww,2.0
2,9874,photo,78.0,en,True,1,7,False,en,3.0
3,43458,photo,43.0,ru,False,8,12,False,ww,2.0
4,9000,photo,76.0,und,True,2,9,False,en,3.0


In [340]:
meme_df.drop(['content_lang', 'caption_lang', 'is_broken'], axis = 1, inplace = True)
meme_df.is_available.value_counts()

True     42837
False    18679
Name: is_available, dtype: int64

In [341]:
meme_df.isna().sum()

meme_id            0
meme_type          0
source_id       1877
is_available       0
likes              0
dislikes           0
language_id      297
dtype: int64

In [342]:
meme_df = pd.get_dummies(meme_df, columns = ['meme_type'])

In [343]:
result = meme_df['is_available'].astype(int)
meme_df['is_available'] = result
del result

In [344]:
meme_df.source_id.value_counts()

75.0     4885
78.0     3815
76.0     2648
4.0      2508
80.0     2471
         ... 
128.0       6
93.0        6
142.0       6
144.0       4
107.0       3
Name: source_id, Length: 142, dtype: int64

In [345]:
nans = meme_df['source_id'].isna() # пропорциональное распределение источников по nan значениям
source_ids = meme_df.source_id.unique()
source_ids = source_ids[np.isnan(source_ids) == False]
length = sum(nans)
replacement = random.choices(source_ids, k=length)
meme_df.loc[nans,'source_id'] = replacement

In [346]:
meme_df.source_id.value_counts()

75.0     4906
78.0     3824
76.0     2662
4.0      2521
80.0     2481
         ... 
127.0      20
103.0      19
144.0      19
107.0      18
128.0      16
Name: source_id, Length: 142, dtype: int64

In [347]:
meme_df.isna().sum()

meme_id                  0
source_id                0
is_available             0
likes                    0
dislikes                 0
language_id            297
meme_type_animation      0
meme_type_photo          0
meme_type_video          0
dtype: int64

In [349]:
meme_df.language_id.value_counts()

3.0    32151
2.0    22094
1.0     6336
4.0      638
Name: language_id, dtype: int64

In [350]:
nans = meme_df['language_id'].isna() # пропорциональное распределение языков по nan значениям
language_ids = meme_df.language_id.unique()
language_ids = language_ids[np.isnan(language_ids) == False]
length = sum(nans)
replacement = random.choices(language_ids, k=length)
meme_df.loc[nans,'language_id'] = replacement

In [352]:
meme_df.language_id.value_counts()

3.0    32228
2.0    22158
1.0     6411
4.0      719
Name: language_id, dtype: int64

In [353]:
meme_df.isna().sum()

meme_id                0
source_id              0
is_available           0
likes                  0
dislikes               0
language_id            0
meme_type_animation    0
meme_type_photo        0
meme_type_video        0
dtype: int64

In [354]:
meme_df.head() # готовый датафрейм

,meme_id,source_id,is_available,likes,dislikes,language_id,meme_type_animation,meme_type_photo,meme_type_video
0,28997,79.0,1,17,14,3.0,0,1,0
1,10873,43.0,1,79,74,2.0,0,1,0
2,9874,78.0,1,1,7,3.0,0,1,0
3,43458,43.0,0,8,12,2.0,0,1,0
4,9000,76.0,1,2,9,3.0,0,1,0
